# Home Dataset Creation

This notebook is used to create the home dataset, by composing different appliances and their respective power consumption. The dataset used is GREEND. The power consumption and metadata are read from the dataset using the [nilmtk](https://github.com/nilmtk) library.

## Setup

In [ ]:
%pip install git+https://github.com/nilmtk/nilmtk # Install from git because the conda package is out of date
%pip install git+https://github.com/nilmtk/nilm_metadata 

## GREEND

In [1]:
from nilmtk import DataSet
from nilmtk.utils import print_dict

greend = DataSet('data/greend.h5')

In [121]:
appliances = {
    1: ['fridge', 'lamp', 'television'],
    3: ['microwave', 'washing machine', 'audio amplifier'],
}

def replace_with_average(group):
    group['power'] = group['power'].mean()
    return group.iloc[:1]

for building, appliance_list in appliances.items():
    print("> Building:", building)
    for appliance in appliance_list:
        print(f"  - Appliance: {appliance} ... ", end="")
        name_undercase = appliance.replace(' ', '_')

        df = next(greend.buildings[building].elec[appliance].load()).reset_index()
        df.columns = ["timestamp", "power"]
        df.drop(columns=["timestamp"], inplace=True)
        #df = df.groupby(df.index // 60).apply(replace_with_average)
        #df = df.reset_index(drop=True)
        df = df.rolling(60, min_periods=1).mean()[::60]

        df.to_csv(f"appliances/{name_undercase}.csv", index=False)
        print("OK")

> Building: 1
  - Appliance: dish washer ... OK
  - Appliance: washing machine ... OK
  - Appliance: radio ... OK
  - Appliance: lamp ... OK
  - Appliance: fridge ... OK
  - Appliance: television ... OK
> Building: 3
  - Appliance: microwave ... OK
> Building: 6
  - Appliance: electric oven ... OK


In [36]:
import numpy as np

activations = greend.buildings[3].elec['microwave'].get_activations(min_off_duration=20, min_on_duration=10, on_power_threshold=20)

lens = [len(a) for a in activations]
lens_mean = np.mean(lens)
print(len(activations), lens_mean)

2311 471.80571181306794


In [206]:
import os 

activations ={
    1: {'fridge': [10,10,40], 'lamp': [10,10,5], 'television': [10,10,10]},
    3: {'microwave': [20,10,20], 'washing machine': [30,30,20], 'audio amplifier': [5,5,10]},
}

for building, appliance_list in activations.items():
    print("> Building:", building)
    for appliance, params in appliance_list.items():
        print(f"  - Appliance: {appliance} ... ", end="")
        name_undercase = appliance.replace(' ', '_')

        #app = pd.read_csv(f"appliances/{name_undercase}.csv"
        app = greend.buildings[building].elec[appliance].get_activations(min_off_duration=params[0], min_on_duration=params[1], on_power_threshold=params[2])

        os.mkdir(f"activations/{name_undercase}")

        for i, activ in enumerate(app):
            activ.reset_index(drop=True, inplace=True)
            activ.dropna(inplace=True)
        
            activ.to_csv(f"activations/{name_undercase}/{i}.csv", index=False)

        print("OK")

> Building: 1
  - Appliance: television ... 

/tmp/ipykernel_8858/684010811.py:22: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  activ.to_csv(f"activations/{name_undercase}/{i}.csv", index=False)


OK


### Extract Appliances

## Deddiag

In [1]:
from nilmtk import DataSet

deddiag = DataSet('data/deddiag.h5')
elec = deddiag.buildings[8].elec
dish_washer = elec['washing machine']

In [13]:
import numpy as np

activations = dish_washer.get_activations(min_on_duration=30, min_off_duration=10, on_power_threshold=10)

lens = [len(a) for a in activations]
lens_mean = np.mean(lens)
print(len(activations), lens_mean)

316 947.2025316455696


## AMPDs2

In [2]:
from nilmtk import DataSet

ampds = DataSet('ampds2/AMPds2.h5')

In [8]:
ampds.buildings[1].elec

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='AMPds', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='AMPds', appliances=[Appliance(type='light', instance=1)])
  ElecMeter(instance=3, building=1, dataset='AMPds', appliances=[Appliance(type='light', instance=2)])
  ElecMeter(instance=4, building=1, dataset='AMPds', appliances=[Appliance(type='light', instance=3)])
  ElecMeter(instance=5, building=1, dataset='AMPds', appliances=[Appliance(type='unknown', instance=1)])
  ElecMeter(instance=6, building=1, dataset='AMPds', appliances=[Appliance(type='unknown', instance=2)])
  ElecMeter(instance=7, building=1, dataset='AMPds', appliances=[Appliance(type='sockets', instance=1)])
  ElecMeter(instance=8, building=1, dataset='AMPds', appliances=[Appliance(type='unknown', instance=3)])
  ElecMeter(instance=9, building=1, dataset='AMPds', appliances=[Appliance(type='unknown', instance=4)])
  ElecMeter(instance=10, building=1, dataset='AMPds', appliances=[A